# Report 2: Theft Over Open Data (TOOD)

> This file is intended to be used as references for sections 3 - 5 which all involves model training, 
> testing, and deployment of the model. Creativity and innovation are encouraged for this and other 
> sections moving forward.

**What is the purpose of this file?**

This notebook will be covering parts 3-5 of the assignment namely going over the following:
1. Predictive Model Building - Building the Predictive Model using Modules within `sklearn`.
2. Model Scoring and Evaluation - Evaluation and Scoring of the Model with Training Data.
3. Model Deployment - Deployment of the model as `.pkl` files.

**NOTE**: The naming of this notebook is intentionally, as it stands for the following:
- `c309` - This is the course code COMP309.
- `r2` - This is the report number which is report 2 of the group project.
- `toodu` - This is the name of the dataset we named and will continue working with in this notebook.
- `model` - This is just a generic name but this notebook will contain sections 3 - 5 which all involves model training, testing, and deployment of the model.

**NOTE**: Provided below is a notebook that includes the above sections, thoroughly covering all aspects of sections 3-5. When required, there will be additional informaiton and insights throughout the notebook to help understand the specifics of the model, algorithms used, and how we handle imbalanced data when training the model.

In [21]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
DEFAULT_DATA_PATH = os.path.join(os.pardir, "data")

# NOTE: Target Feature is: PREMISES_TYPE
target = "PREMISES_TYPE"

# NOTE: This is probably going to take a little bit of testing to get the right features we want.
features = [
    "PREMISES_TYPE",
    "LOCATION_TYPE",
    "UCR_CODE",
    "UCR_EXT",
    "OFFENCE",
    "MCI_CATEGORY",
    "HOOD_158",
    "NEIGHBOURHOOD_158",
    "HOOD_140",
    "NEIGHBOURHOOD_140",
    "LONG_WGS84",
    "LAT_WGS84"
]

toodu_ft_df = pd.read_csv(os.path.join(DEFAULT_DATA_PATH, "Theft_Over_Open_Data_Cleaned.csv"))

In [25]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

le = LabelEncoder()
scaler = StandardScaler()

le.fit(toodu_ft_df[target])
toodu_ft_df[target] = le.transform(toodu_ft_df[target])

scaler.fit(toodu_ft_df[features])
toodu_ft_df[features] = scaler.transform(toodu_ft_df[features], columns_to_fill = features)

ValueError: could not convert string to float: 'Other Commercial / Corporate Places (For Profit, Warehouse, Corp. Bldg'

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import cross_val_score, cross_val_predict


x = toodu_ft_df[features]
y = toodu_ft_df[target]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
# x_train_processed
# y_train_processed

In [24]:
from sklearn.feature_selection import SelectFromModel, mutual_info_regression
from sklearn.metrics import accuracy_score, precision_score, average_precision_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix

In [ ]:
# TODO: Training the Model here.

In [ ]:
# TODO: Add Logistic Regression Model here.

In [ ]:
# TODO: Add Decision Tree Model here.

In [ ]:
# TODO: Add Combination of Logistic Regression and Decision Tree Model here.